In [2]:
import json
from itertools import cycle, islice
from shapely.geometry import LineString, Polygon
import numpy as np
from basepolygon_class import GoalPolygon
from itertools import combinations

'''f = open("/Users/andrewastakhov/dumps/pl.json", "r")
points = json.load(f)['pts']'''

'f = open("/Users/andrewastakhov/dumps/pl.json", "r")\npoints = json.load(f)[\'pts\']'

In [3]:
place = GoalPolygon('Жилой комплекс «Одинцово-1»')
v = place.polygon
s = v.simplify(10, preserve_topology=True)
coor_list = (list(s.boundary.coords[0:len(s.boundary.coords) - 1]))
poly = Polygon(s)

/opt/anaconda3/envs/tests/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/anaconda3/envs/tests/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/anaconda3/envs/tests/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [10]:
def vector_search(point_list, key):
    cr = []
    n = list(islice(cycle(range(len(point_list))), 0, len(point_list) + 2))
    for i in range(len(point_list)):
        v_one = [point_list[n[i]][0] - point_list[n[i + 1]][0], point_list[n[i]][1] - point_list[n[i + 1]][1], 0]
        v_two = [point_list[n[i + 1]][0] - point_list[n[i + 2]][0], point_list[n[i + 1]][1] - point_list[n[i + 2]][1],
                 0]
        v_o_norm, v_t_norm = v_one / np.linalg.norm(v_one), v_two / np.linalg.norm(v_two)
        if np.cross(v_o_norm, v_t_norm)[2] < key:
            cr.append(n[i + 1])
        else:
            pass
    cross = [*cr, cr[0] + cr[-1] + 1]
    double_points = [*range(len(point_list)), *range(len(point_list))]
    segments = [double_points[cross[c]:cross[c + 1]] for c in range(len(cross) - 1)]
    return segments


def clusterize(poly_edges):
    lines = []
    vectors_list = []
    key_vals = vector_search(poly_edges.boundary.coords[0:len(s.boundary.coords) - 1], 0.7)
    vertices = poly_edges.boundary.coords[0:len(s.boundary.coords)]
    for k in range(len(vertices) - 1):
        lines.append(LineString([vertices[k], vertices[k + 1]]))
    for i in key_vals:
        l=[]
        for ii in i:
            vect = [lines[ii].coords[0][0] - lines[ii].coords[1][0],
                    lines[ii].coords[0][1] - lines[ii].coords[1][1]]
            l.append(vect)
        vectors_list.append(l)

    return vectors_list


In [11]:
v = clusterize(poly)
print(v)

[[1, 2, 3], [4, 5, 6], [7], [8, 9], [10, 11, 12], [13, 0]]
[[[40.27832240751013, 17.184793210588396], [-90.76909511670237, 342.689586349763], [-597.2236579991877, -170.50443049520254]], [[-302.8780582649051, -317.03347889892757], [308.1424075482064, -308.39204586390406], [204.62842559075216, 203.57871970254928]], [[278.06133277650224, 43.473096154630184]], [[101.89771133306203, -376.15734403952956], [63.9330358550651, 17.928993717767298]], [[9.30055009451462, -33.0232094200328], [65.70811673195567, 18.42902405373752], [-62.45231896225596, 221.67703934758902]], [[72.29789360892028, 20.2681416394189], [-90.92466560343746, 319.88111454155296]]]


In [12]:
from scipy.linalg import svd
import numpy as np
#A = np.asarray(points)

NameError: name 'points' is not defined

Пример с одной матрицей

In [10]:
a = np.asarray(A[0])

In [11]:
U, s, Vh=svd(a)

Получить Sigma из s Хуй знает зачем сайпи говорят что это полезно

In [12]:
sigma = np.zeros(a.shape)
for i in range(min(a.shape)):
            sigma[i, i] = s[i]

А точно, как минимум для обратной проверки, "B" должно быть равно "a"

In [15]:
B = U @ sigma @ Vh

In [18]:
a

array([[367.5282953 , 411.27996727],
       [411.27996727, 570.2315431 ],
       [570.2315431 , 439.81468985]])

In [19]:
B

array([[367.5282953 , 411.27996727],
       [411.27996727, 570.2315431 ],
       [570.2315431 , 439.81468985]])

Функция итерирующая список матриц и вычисляющая свд для каждой

In [21]:
def csvd(aa):
    svdd=[]
    for j, a_ in enumerate(aa):

        a=np.asarray(a_)
        print(f"A:\n{a}")
        U, s, Vh=svd(a)
        print("U")
        print(U)
        sigma = np.zeros(a.shape)
        print("s")
        print(s)
        print("Vh")
        print(Vh)
        for i in range(min(a.shape)):
            sigma[i, i] = s[i]
        print("Sigma")
        print(sigma)
        svdd.append((a, U.tolist(), s.tolist(), Vh, sigma))


    return svdd

In [22]:
n=csvd(A)

A:
[[367.5282953  411.27996727]
 [411.27996727 570.2315431 ]
 [570.2315431  439.81468985]]
U
[[-0.48428925 -0.12512533 -0.8659143 ]
 [-0.61181407 -0.65905685  0.43741012]
 [-0.62541783  0.74161157  0.24262071]]
s
[1138.29645279  146.37500898]
Vh
[[-0.69072503 -0.72311751]
 [ 0.72311751 -0.69072503]]
Sigma
[[1138.29645279    0.        ]
 [   0.          146.37500898]
 [   0.            0.        ]]
A:
[[439.81468985 606.41515894]
 [606.41515894 428.1395325 ]
 [428.1395325  139.62391838]]
U
[[-0.65213419  0.67449093  0.34609678]
 [-0.66082103 -0.28201777 -0.69554406]
 [-0.37153272 -0.68229609  0.62963122]]
s
[1118.05423503  254.89632419]
Vh
[[-0.75722439 -0.65315483]
 [-0.65315483  0.75722439]]
Sigma
[[1118.05423503    0.        ]
 [   0.          254.89632419]
 [   0.            0.        ]]
A:
[[139.62391838 193.96186923]]
U
[[-1.]]
s
[238.98963429]
Vh
[[-0.58422583 -0.81159114]
 [-0.81159114  0.58422583]]
Sigma
[[238.98963429   0.        ]]
A:
[[193.96186923 521.81483578]
 [521.814835